In [2]:
import random
import gym
import numpy as np
from collections import deque

In [3]:
from sklearn.multioutput import MultiOutputRegressor
from lightgbm import LGBMRegressor
from scores.score_logger import ScoreLogger
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'scores'

In [8]:
#seu jogo de escolha
ENV_NAME = "CartPole-v1"
GAMMA = 0.95
LEARNING_RATE = 0.001
MEMORY_SIZE = 1000
BATCH_SIZE = 20
EXPLORATION_MAX = 1.0
EXPLORATION_MIN = 0.05
EXPLORATION_DECAY = 0.96

In [44]:
##def cartpole
env = gym.make(ENV_NAME)
observation_space = env.observation_space.shape[0]
action_space = env.action_space.n

In [5]:
class DQNSolver:
    def __init__(self, observation_space, action_space):
        
        self.exploration_rate = EXPLORATION_MAX
        self.action_space = action_space
        self.memory = deque(maxlen = MEMORY_SIZE)        
        self.model = Sequential()
        self.model.add(Dense(24, input_shape=(observation_space,), activation="relu"))
        self.model.add(Dense(24, activation="relu"))
        self.model.add(Dense(self.action_space, activation="linear"))
        self.model.compile(loss="mse", optimizer=Adam(lr=LEARNING_RATE))

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_stage, done))
    
    def act(self, state):
        if np.random.rand()< exploration_rate:
            return random.randrange(self.action_space)
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])
    
    def exploration_rate(self):
        if len(self.memory)<BATCH_SIZE:
            return
        batch = random.sample(self.memory, BATCH_SIZE)
        for state, action, reward, state_next, terminal in batch:
            q_update = reward
            if not terminal:
                q_update =  (reward + GAMMA * np.amax(self.model.predict(state_next)[0]))
            q_values = self.model.predict(state)
            q_values[0][action] = q_update
            self.model.fit(state, q_values, verbose = 0)
        self.exploration_rate *= EXPLORATION_DECAY
        self.exploration_rate = max(EXPLORATION_MIN, self.exploration_rate)

In [1]:
###########################################################################################

In [54]:
env = gym.make('MountainCarContinuous-v0') # try for different environements
env.action_space

Box(1,)

In [78]:
env.reset()

array([-0.03570481,  0.01183121,  0.0260376 ,  0.03959878])

In [1]:
# #https://gist.github.com/iambrian/2bcc8fc03eaecb2cbe53012d2f505465
# import gym
# env = gym.make('CartPole-v0')
# highscore = 0
# for i_episode in range(20): # run 20 episodes
#     observation = env.reset()
#     points = 0 # keep track of the reward each episode
#     while True: # run until episode is done
#         env.render()
#         action = 1 if observation[2] > 0 else 0 # if angle if positive, move right. if angle is negative, move left
#         observation, reward, done, info = env.step(action)
#         points += reward
#         if done:
#           if points > highscore: # record high score
#             highscore = points
#             break
            
# env.close()

In [103]:
import gym
env = gym.make('CartPole-v0')
for i_episode in range(20):
    observation = env.reset() # initial observation
    aux = 0
    for t in range(50):
        env.render()
        aux += 1
        print('{}.observation: {}'.format(aux, observation))
        action = env.action_space.sample()
        print('--ação:{}'.format(action))
        observation, reward, done, info = env.step(action)
        #ok onde está as condições de done?
        print('--observation:{}, reward:{}, done:{}, info:{}'.format(observation, reward, done, info))
        if done:
            print("\n")
            print("Episode finished after {} timesteps".format(t+1).upper())
            break
env.close()

In [102]:
env.step(action)

(array([ -9.13693864, -12.93749645,   4.91046668,  -2.18595879]),
 0.0,
 True,
 {})

In [1]:
#########################################################################################

In [7]:
import numpy as np
import gym
import random
import time
from IPython.display import clear_output

In [8]:
env = gym.make('FrozenLake-v0')

In [9]:
action_space = env.action_space.n
state_space = env.observation_space.n

In [10]:
q_table = np.zeros((state_space, action_space))

In [22]:
num_episodes = 10000
max_steps = 150

learning_rate = 0.1
discount_rate = 0.95

exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001

In [23]:
reward_episodes_list = []

for episode in range(num_episodes):
    state = env.reset()
    #env.render()
    done = False
    rewards_current_episode = 0
    
    for step in range(max_steps):
        
        exploration_rate_threshold = random.uniform(0,1)
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state,:])
        else:
            action = env.action_space.sample()
        
        new_state, reward, done, info = env.step(action)
        
        q_table[state, action] = q_table[state, action] * (1 - learning_rate) +\
        learning_rate*(reward + discount_rate*np.max(q_table[new_state, :]))
        
        state = new_state
        rewards_current_episode += reward
        
        if done == True:
            break
    
    exploration_rate = min_exploration_rate + (max_exploration_rate - min_exploration_rate)* \
        np.exp(-exploration_decay_rate*episode)
    
    reward_episodes_list.append(rewards_current_episode)

reward_per_k = np.split(np.array(reward_episodes_list), num_episodes/1000)
count = 1000
print('--'*10)
for r in reward_per_k:
    print(count, ': ', str(sum(r/1000)))
    count += 1000
    
print('--'*10)
print(q_table)

--------------------
1000 :  0.048000000000000036
2000 :  0.18600000000000014
3000 :  0.34800000000000025
4000 :  0.4850000000000004
5000 :  0.5940000000000004
6000 :  0.6500000000000005
7000 :  0.5970000000000004
8000 :  0.6100000000000004
9000 :  0.6420000000000005
10000 :  0.6830000000000005
--------------------
[[0.15395433 0.15133235 0.16772163 0.15368095]
 [0.09034392 0.08680864 0.05423558 0.16323595]
 [0.18362902 0.10506937 0.11097827 0.09712907]
 [0.07692466 0.07705253 0.05608361 0.09707123]
 [0.20396458 0.13590793 0.13583458 0.16676089]
 [0.         0.         0.         0.        ]
 [0.06752913 0.09046227 0.2698702  0.03622247]
 [0.         0.         0.         0.        ]
 [0.15837212 0.17502338 0.15160105 0.2415143 ]
 [0.31607278 0.35896916 0.25476007 0.20933769]
 [0.45711262 0.31191563 0.30606706 0.26553679]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.30892439 0.44177871 0.5213282  0.27047843]
 [0.52930222 0.76875506 0

In [24]:
import gym
env = gym.make("Taxi-v2")

In [27]:
env.reset()
env.observation_space.n
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [64]:
state = env.reset()
counter = 0
reward = None
while reward != 20:
    state, reward, done, info = env.step(env.action_space.sample())
    counter += 1

print(counter)


6614


In [72]:
import numpy as np
Q = np.zeros([env.observation_space.n, env.action_space.n])

G = 0
alpha = 0.618

for episode in range(1,1001):
    done = False
    G, reward = 0,0
    state = env.reset()
    while done != True:
            action = np.argmax(Q[state]) #1
            state2, reward, done, info = env.step(action) #2
            Q[state,action] += alpha * (reward + np.max(Q[state2]) - Q[state,action]) #3
            G += reward
            state = state2   
    if episode % 50 == 0:
        print('Episode {} Total Reward: {}'.format(episode,G))

Episode 50 Total Reward: -117
Episode 100 Total Reward: 8
Episode 150 Total Reward: -76
Episode 200 Total Reward: -20
Episode 250 Total Reward: -1
Episode 300 Total Reward: 9
Episode 350 Total Reward: 11
Episode 400 Total Reward: -7
Episode 450 Total Reward: 10
Episode 500 Total Reward: 10
Episode 550 Total Reward: 2
Episode 600 Total Reward: 5
Episode 650 Total Reward: 6
Episode 700 Total Reward: 8
Episode 750 Total Reward: 10
Episode 800 Total Reward: 6
Episode 850 Total Reward: 7
Episode 900 Total Reward: 11
Episode 950 Total Reward: 9
Episode 1000 Total Reward: 7


In [2]:
import gym
env = gym.make("MsPacman-v0")
state = env.reset()
env.render()
env.action_space.n
env.env.get_action_meanings()
state = env.reset()
reward, info, done = None, None, None
while done != True:
    state, reward, done, info = env.step(env.action_space.sample())
    env.render()

In [3]:
import numpy as np
import math
import random
from matplotlib import pyplot as plt
from IPython.display import clear_output

PI = 3.1415926
e = 2.71828

In [6]:
def get_rand_number(min_value, max_value):
    """
    This function gets a random number from a uniform distribution between
    the two input values [min_value, max_value] inclusively
    Args:
    - min_value (float)
    - max_value (float)
    Return:
    - Random number between this range (float)
    """
    range = max_value - min_value
    choice = random.uniform(0,1)
    return min_value + range*choice
def f_of_x(x):
    """
    This is the main function we want to integrate over.
    Args:
    - x (float) : input to function; must be in radians
    Return:
    - output of function f(x) (float)
    """
    return (e**(-1*x))/(1+(x-1)**2)

In [7]:
def crude_monte_carlo(num_samples=5000):
    """
    This function performs the Crude Monte Carlo for our
    specific function f(x) on the range x=0 to x=5.
    Notice that this bound is sufficient because f(x)
    approaches 0 at around PI.
    Args:
    - num_samples (float) : number of samples
    Return:
    - Crude Monte Carlo estimation (float)
    
    """
    lower_bound = 0
    upper_bound = 5
    
    sum_of_samples = 0
    for i in range(num_samples):
        x = get_rand_number(lower_bound, upper_bound)
        sum_of_samples += f_of_x(x)
    
    return (upper_bound - lower_bound) * float(sum_of_samples/num_samples)